In [92]:
# Check if CUDA is available and set the device accordingly
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [93]:
import os
from glob import glob
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt

In [94]:
# Define the paths to the images
train_images_dir = "C:/Users/IMRAN/Partion 2/UNHAS/MKPK/KP/WEEK3/Dataset/ISIC_2019_Training_Input"
test_images_dir = "C:/Users/IMRAN/Partion 2/UNHAS/MKPK/KP/WEEK3/Dataset/ISIC_2019_Test_Input"

# Load the metadata and ground truth CSV files using pandas
train_metadata = pd.read_csv("C:/Users/IMRAN/Partion 2/UNHAS/MKPK/KP/WEEK3/Dataset/ISIC_2019_Training_Metadata.csv")
train_groundtruth = pd.read_csv("C:/Users/IMRAN/Partion 2/UNHAS/MKPK/KP/WEEK3/Dataset/ISIC_2019_Training_GroundTruth.csv")
test_metadata = pd.read_csv("C:/Users/IMRAN/Partion 2/UNHAS/MKPK/KP/WEEK3/Dataset/ISIC_2019_Test_Metadata.csv")

In [95]:
# Display first few rows of each DataFrame
print("Training Metadata:")
print(train_metadata.head())

print("\nTraining GroundTruth:")
print(train_groundtruth.head())

Training Metadata:
          image  age_approx anatom_site_general lesion_id     sex
0  ISIC_0000000        55.0      anterior torso       NaN  female
1  ISIC_0000001        30.0      anterior torso       NaN  female
2  ISIC_0000002        60.0     upper extremity       NaN  female
3  ISIC_0000003        30.0     upper extremity       NaN    male
4  ISIC_0000004        80.0     posterior torso       NaN    male

Training GroundTruth:
          image  MEL   NV  BCC   AK  BKL   DF  VASC  SCC  UNK
0  ISIC_0000000  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
1  ISIC_0000001  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
2  ISIC_0000002  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
3  ISIC_0000003  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
4  ISIC_0000004  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0


In [96]:
# Merge metadata with ground truth on the 'image' column
train_data = pd.merge(train_metadata, train_groundtruth, on='image')

# Display first few rows of the merged DataFrame
print("\nMerged Training Data:")
print(train_data.head())



Merged Training Data:
          image  age_approx anatom_site_general lesion_id     sex  MEL   NV  \
0  ISIC_0000000        55.0      anterior torso       NaN  female  0.0  1.0   
1  ISIC_0000001        30.0      anterior torso       NaN  female  0.0  1.0   
2  ISIC_0000002        60.0     upper extremity       NaN  female  1.0  0.0   
3  ISIC_0000003        30.0     upper extremity       NaN    male  0.0  1.0   
4  ISIC_0000004        80.0     posterior torso       NaN    male  1.0  0.0   

   BCC   AK  BKL   DF  VASC  SCC  UNK  
0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0   0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0   0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0   0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0   0.0  0.0  0.0  


In [97]:
# Display the shape of the merged DataFrame
print(f"Shape of the merged DataFrame: {train_data.shape}")

Shape of the merged DataFrame: (25331, 14)


In [98]:
# Apply necessary transformations and augmentations
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
#from keras.applications import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os

# Define the paths to the images
train_images_dir = "C:/Users/IMRAN/Partion 2/UNHAS/MKPK/KP/WEEK3/Dataset/ISIC_2019_Training_Input"
test_images_dir = "C:/Users/IMRAN/Partion 2/UNHAS/MKPK/KP/WEEK3/Dataset/ISIC_2019_Test_Input"

# Ensure the image filenames have the correct format (append .jpg if necessary)
train_data['image'] = train_data['image'].apply(lambda x: x + '.jpg')

# Create the full path for each image
train_data['image_path'] = train_data['image'].apply(lambda x: os.path.join(train_images_dir, x))

# Split the data into training and validation sets
train_df, val_df = train_test_split(train_data, test_size=0.2, stratify=train_data[['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']])

# ImageDataGenerator for training and validation with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create the training and validation generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,  # Images are already included in image_path column
    x_col='image_path',
    y_col=['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC'],
    target_size=(224, 224),  # Resize images to match VGG16 input size
    batch_size=32,
    class_mode='raw',  # For multi-output regression/classification
    shuffle=True
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=None,  # Images are already included in image_path column
    x_col='image_path',
    y_col=['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC'],
    target_size=(224, 224),  # Resize images to match VGG16 input size
    batch_size=32,
    class_mode='raw',  # For multi-output regression/classification
    shuffle=False
)

Found 20264 validated image filenames.
Found 5067 validated image filenames.


In [99]:
print(f"Training samples: {train_generator.n}")
print(f"Validation samples: {validation_generator.n}")

print(f"Training steps per epoch: {len(train_generator)}")
print(f"Validation steps: {len(validation_generator)}")

Training samples: 20264
Validation samples: 5067
Training steps per epoch: 634
Validation steps: 159


In [100]:
x_batch, y_batch = next(train_generator)
print(f"x_batch shape: {x_batch.shape}")
print(f"y_batch shape: {y_batch.shape}")

x_batch shape: (32, 224, 224, 3)
y_batch shape: (32, 8)


In [101]:
# Function to load images and convert to arrays when needed
def load_image_from_path(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    return img_array / 255.0  # Normalize pixel values to [0, 1]

In [102]:
# Keras Data Generator function that yields image arrays from DataFrame
def data_generator(dataframe, batch_size=32, target_size=(224, 224)):
    num_samples = len(dataframe)
    while True:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_data = dataframe.iloc[offset:offset+batch_size]
            images = []
            labels = []

            for _, row in batch_data.iterrows():
                img_array = load_image_from_path(row['image_path'], target_size)
                images.append(img_array)

                # Collect the ground truth label for this image (you might need to adjust label format)
                labels.append(row['benign_malignant'])  # Example: assuming label is in 'benign_malignant' column

            X_train = tf.convert_to_tensor(images)
            y_train = tf.convert_to_tensor(labels)
            yield X_train, y_train

# Initialize the data generator
batch_size = 32
train_generator = data_generator(train_data, batch_size=batch_size)

# Define a simple example model (replace with your DenseNet or other model)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification example
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with the generator
steps_per_epoch = len(train_data) // batch_size
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10)

KeyError: 'benign_malignant'

In [88]:
# Define the number of classes
num_classes = 8

# Load the InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom classification head
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_379 (Conv2D)            (None, 111, 111, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_376 (Batch  (None, 111, 111, 32  96         ['conv2d_379[0][0]']             
 Normalization)                 )                                                           

In [91]:
# train the model
history = model.fit(
    train_generator,
    steps_per_epoch=(len(train_data) // 32),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=(len(val_df) // 32)
)


StopIteration: 

In [25]:
model.save("InceptionV3FT.h5")